In [6]:
def plot_skyplot_contour(gdf) :
    
    plt.rcdefaults() 
    sns.set_theme()
    
    # folder
    # ===================================================
    if not os.path.exists('data/FigurePlot/plot_skyplot_contour' ):
        os.makedirs( 'data/FigurePlot/plot_skyplot_contour' )
        print('create directory : skyplot_contour' )
        
    folder_path = 'data/FigurePlot/plot_skyplot_contour/'

    # calculation 
    # ===================================================
    def apply_contour(r):
        if r['State'] == 'LOS' : return 2 
        if r['State'] == 'NLOS': return 1
        else : return 0
    # 以 apply_fx 建立新的列（Column）
    gdf['contour'] = gdf.apply(apply_contour, axis=1)

    def apply_fx(r):
        output = math.radians(r['SAT_AZ'])
        return output
    # 以 apply_fx 建立新的列（Column）
    gdf['radians_AZ'] = gdf.apply(apply_fx, axis=1)

    
    
    buffer_df = gdf
    
    location_list = buffer_df["location"].unique()
    for location in location_list :
    
        gdf = buffer_df[buffer_df['location']==location]
        
        # fliter
        # ===================================================
        fliter_LOS  = (gdf['State']=='LOS')
        fliter_NLOS = (gdf['State']=='NLOS')
        fliter_OFR = (gdf['State']=='out_of_range')

        # plot
        # ===================================================
        fig = plt.figure(figsize=(10,10))
        plt.xticks([])
        plt.yticks([])

        #name_split = name.split('_')
        #plt.title('Skyplots for contour, date : ' + name_split[3] + ', location : ' + name_split[2] + ', satellite amount : ' + name_split[4], fontsize = 15)

        """
        ax = fig.add_subplot(121)
        ax.tricontourf(gdf['SAT_AZ'], 90-gdf['SAT_EL'], gdf['contour'], levels=2)
        """
        gdf_NLOS = gdf[fliter_NLOS]
        gdf_LOS = gdf[fliter_LOS]
        gdf_OFR = gdf[fliter_OFR]

        ax = fig.add_subplot(111)
        theta = np.radians(gdf['SAT_AZ']) - math.pi/2
        rad = 90-gdf['SAT_EL']
        ax.tricontourf(rad * np.cos(theta), -1 *  rad * np.sin(theta), gdf['contour'], levels=2, alpha = 1)

        """
        theta = np.radians(gdf_NLOS['SAT_AZ']) - (math.pi/2)
        rad = 90-gdf_NLOS['SAT_EL']
        ax.scatter(rad * np.cos(theta), -1 *  rad * np.sin(theta), s=5, c= '#C44E52') # '#C44E52'

        theta = np.radians(gdf_LOS['SAT_AZ']) - (math.pi/2)
        rad = 90-gdf_LOS['SAT_EL']
        ax.scatter(rad * np.cos(theta), -1 *  rad * np.sin(theta), s=5, c= '#55A868') # '#C44E52'

        theta = np.radians(gdf_OFR['SAT_AZ']) - (math.pi/2)
        rad = 90-gdf_OFR['SAT_EL']
        ax.scatter(rad * np.cos(theta), -1 *  rad * np.sin(theta), s=5, c= '#4C72B0') # '#C44E52'
        """

        #ax.tricontourf(gdf['radians_AZ'], gdf['SAT_EL'], gdf['contour'], levels=2, cmap='hsv')

        ax.set_aspect('equal')
        ax.axis('off')

        ax_polar = fig.add_axes(ax.get_position(), projection='polar') #polar=True)
        ax_polar.set_facecolor('none') # make transparent
        ax_polar.set_ylim(0,rad.max())


        ax_polar.set_theta_direction(-1)
        ax_polar.set_theta_zero_location('N')
        ax_polar.set_xticklabels(['N', '', 'E', '', 'S', '', 'W', ''])
        ax_polar.tick_params(axis='both', which='major', labelsize=30)

        ax_polar.set_rlim(90,0)
        ax_polar.set_rticks([90,60,30,0])  

        red_patch = mpatches.Patch(color='#F6AB83', label='LOS')#C44E52
        blue_patch = mpatches.Patch(color='#3F1B43', label='out of range')#4C72B0
        green_patch = mpatches.Patch(color='#CB1B4F', label='NLOS')#55A868
        ax.legend(handles=[green_patch,red_patch,blue_patch],loc='upper left', prop={'size': 20})


        plt.savefig(folder_path + 'contour ' + location + '.png')
        plt.close()
